In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load gdsc2


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/240 [00:00<?, ?it/s]

epoch:   0 loss:0.689846 auc:0.5452
epoch:  20 loss:0.153458 auc:0.7654
epoch:  40 loss:0.137307 auc:0.7332
epoch:  60 loss:0.123401 auc:0.7200
epoch:  80 loss:0.114666 auc:0.7002
epoch: 100 loss:0.108796 auc:0.7009
epoch: 120 loss:0.103024 auc:0.7725
epoch: 140 loss:0.097316 auc:0.8232
epoch: 160 loss:0.093166 auc:0.8522
epoch: 180 loss:0.089064 auc:0.8796
epoch: 200 loss:0.088634 auc:0.8827
epoch: 220 loss:0.083674 auc:0.9155
epoch: 240 loss:0.081201 auc:0.9280
epoch: 260 loss:0.080571 auc:0.9405
epoch: 280 loss:0.078929 auc:0.9425
epoch: 300 loss:0.077553 auc:0.9389
epoch: 320 loss:0.076606 auc:0.9407
epoch: 340 loss:0.076069 auc:0.9439
epoch: 360 loss:0.078263 auc:0.9498
epoch: 380 loss:0.075264 auc:0.9468
epoch: 400 loss:0.074542 auc:0.9509
epoch: 420 loss:0.078169 auc:0.9474
epoch: 440 loss:0.074534 auc:0.9500
epoch: 460 loss:0.073720 auc:0.9485
epoch: 480 loss:0.073544 auc:0.9479
epoch: 500 loss:0.074370 auc:0.9540
epoch: 520 loss:0.073244 auc:0.9547
epoch: 540 loss:0.072955 auc

  2%|▏         | 4/240 [00:43<42:25, 10.79s/it]

Fit finished.
epoch:   0 loss:0.699163 auc:0.5627
epoch:  20 loss:0.155099 auc:0.6219
epoch:  40 loss:0.138868 auc:0.6906
epoch:  60 loss:0.124582 auc:0.7572
epoch:  80 loss:0.115394 auc:0.8199
epoch: 100 loss:0.109145 auc:0.8020
epoch: 120 loss:0.103163 auc:0.7797
epoch: 140 loss:0.097371 auc:0.7695
epoch: 160 loss:0.093388 auc:0.7291
epoch: 180 loss:0.088771 auc:0.7550
epoch: 200 loss:0.087863 auc:0.8193
epoch: 220 loss:0.083158 auc:0.7904
epoch: 240 loss:0.082328 auc:0.7447
epoch: 260 loss:0.079631 auc:0.7498
epoch: 280 loss:0.078262 auc:0.7750
epoch: 300 loss:0.077318 auc:0.7889
epoch: 320 loss:0.077150 auc:0.7276
epoch: 340 loss:0.076161 auc:0.7500
epoch: 360 loss:0.081119 auc:0.8313
epoch: 380 loss:0.075333 auc:0.7727
epoch: 400 loss:0.074620 auc:0.7965
epoch: 420 loss:0.074874 auc:0.8142
epoch: 440 loss:0.074542 auc:0.7561
epoch: 460 loss:0.073854 auc:0.7606
epoch: 480 loss:0.076302 auc:0.7039
epoch: 500 loss:0.073885 auc:0.7873
epoch: 520 loss:0.073258 auc:0.7836
epoch: 540 los

  5%|▍         | 11/240 [01:00<18:30,  4.85s/it]

...

epoch: 980 loss:0.079935 auc:0.7626
Fit finished.
epoch:   0 loss:0.703728 auc:0.3192
epoch:  20 loss:0.153014 auc:0.8329
epoch:  40 loss:0.136446 auc:0.7322
epoch:  60 loss:0.122685 auc:0.6500
epoch:  80 loss:0.114335 auc:0.6156
epoch: 100 loss:0.108174 auc:0.6211
epoch: 120 loss:0.102339 auc:0.6257
epoch: 140 loss:0.098102 auc:0.6673
epoch: 160 loss:0.092809 auc:0.6328
epoch: 180 loss:0.088677 auc:0.6725
epoch: 200 loss:0.085796 auc:0.7412
epoch: 220 loss:0.083316 auc:0.7196
epoch: 240 loss:0.082334 auc:0.7916
epoch: 260 loss:0.079817 auc:0.7673
epoch: 280 loss:0.078686 auc:0.7864
epoch: 300 loss:0.077727 auc:0.7821
epoch: 320 loss:0.077170 auc:0.7559
epoch: 340 loss:0.076486 auc:0.7809
epoch: 360 loss:0.075612 auc:0.7935
epoch: 380 loss:0.077105 auc:0.8181
epoch: 400 loss:0.075644 auc:0.7842
epoch: 420 loss:0.074528 auc:0.7867
epoch: 440 loss:0.076149 auc:0.7655
epoch: 460 loss:0.074505 auc:0.7956
epoch: 480 loss:0.073800 auc:0.7898
epoch: 500 loss:0.073580 auc:0.7959
epoch: 520 los

 98%|█████████▊| 235/240 [27:52<00:23,  4.73s/it]

epoch: 980 loss:0.073251 auc:0.8067
Fit finished.
epoch:   0 loss:0.702389 auc:0.5324
epoch:  20 loss:0.153123 auc:0.6594
epoch:  40 loss:0.136516 auc:0.6490
epoch:  60 loss:0.122789 auc:0.7503
epoch:  80 loss:0.114512 auc:0.7090
epoch: 100 loss:0.108720 auc:0.6483
epoch: 120 loss:0.103056 auc:0.6062
epoch: 140 loss:0.097694 auc:0.6337
epoch: 160 loss:0.093253 auc:0.6949
epoch: 180 loss:0.089330 auc:0.7367
epoch: 200 loss:0.086673 auc:0.7488
epoch: 220 loss:0.083810 auc:0.7836
epoch: 240 loss:0.082787 auc:0.7853
epoch: 260 loss:0.080196 auc:0.8037
epoch: 280 loss:0.079158 auc:0.8216
epoch: 300 loss:0.077744 auc:0.8125
epoch: 320 loss:0.076779 auc:0.8002
epoch: 340 loss:0.076816 auc:0.8050
epoch: 360 loss:0.076050 auc:0.8269
epoch: 380 loss:0.075247 auc:0.8191
epoch: 400 loss:0.074888 auc:0.7911
epoch: 420 loss:0.075755 auc:0.8113
epoch: 440 loss:0.074461 auc:0.8161
epoch: 460 loss:0.074029 auc:0.8138
epoch: 480 loss:0.073782 auc:0.8110
epoch: 500 loss:0.074681 auc:0.8193
epoch: 520 los

100%|██████████| 240/240 [28:02<00:00,  7.01s/it]

epoch: 980 loss:0.072337 auc:0.8818
Fit finished.


In [7]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")